# Tutorial

## Introduction

In this tutorial we will explore the functionality of ThermoPot using the
chalcogenide perovskite BaZrS$_3$ as a motivating example. We will consider
three reactions:

(1) BaZrS$_3$ -> BaS + ZrS$_2$

(2) BaZrS$_3$ -> Ba + Zr + 3S

(3) BaZrS$_3$ -> BaS$_2$ + ZrS

We will use ThermoPot to calculate the change in Gibbs free energy for each
of these reactions, at a range of temperature and pressures. We
will also identify at which temperatures and pressures (if any) the
perovskite decomposes into it's competing binary (BaS, BaS2, ZrS, ZrS$_2$) or
elemental (Ba, Zr, S) phases.

To do these calculations we need the total ground-state energy of each
material, calculated using DFT or elsewise. For predictions and finite
temperature and pressure we need, in addition, vibrational data for each
material. For solid materials this can also be calculated from
first-principles using harmonic lattice dynamical theory. For gaseous
compounds (not considered in this tutorial), where there is a strong dependence
 on pressures close to standard pressure, experimental data can be used.

## Code structure

There are three core classes in ThermoPot:

1) Calculation to store data for a
single calculation. For example, an energy calculated using the HSE06 functional)

2) Material to store data and make predictions for a single material. For
example BaZrS$_3$, with energies calculated at
various levels of theory.

3) Reaction to store data  and make predictions for a single chemical
reaction. For example BaZrS$_3$ -> BaS$_2$ + ZrS

The structure is hierachial; one or more `Calculation` instances are used to
build a `Material`, and three or more `Material` instances are used to
calculate a `Reaction`. In addition to this there is the
`Potential` class to store and plot a single  thermodynamic potential, and
the `Potentials` class to store and plot multiple thermodynamic potentials.



## Step 1 - import relevant libraries



In [1]:
from thermopot import calculations, materials, reactions, potentials

## Step 2 - create `calculations`

There are currently two ways to create a `Calculations` instance:

1) Manually input the attributes (calculated energy, xc-functional type,
   volume, number of atoms). 
   
2) Parse a FHI-aims output file to read this data
   automatically. Provide the path for an `aims.out` file. 

We will provide an example for each which uses the `vars`
function to show that the class attributes are equal.

We plan to extend the code so that it can parse more electronic structure
codes. Please [raise an issue on the Github repo](https://github.com/NU-CEM/ThermoPot/issues) if you would like us to
support a particular code.

In [2]:
BaS_calc = calculations.Calculation(volume=63.2552,energy=-235926.586148547,
                            xc='pbesol',
                        NAtoms=2)
vars(BaS_calc)

{'volume': 63.2552,
 'filepath': None,
 'energy': -235926.586148547,
 'xc': 'pbesol',
 'NAtoms': 2}

In [3]:
BaS_calc = calculations.AimsCalculation("../BaZrS3/raw_aims_files/binary/BaS_Fm-3m/pbesol/aims.out")
vars(BaS_calc)

{'volume': 63.2552,
 'filepath': '../BaZrS3/raw_aims_files/binary/BaS_Fm-3m/pbesol/aims.out',
 'energy': -235926.58614863,
 'xc': 'pbesol',
 'NAtoms': 2}

To access in-notebook help for each module, function or class, docstrings can
 be printed by appending `?`. Alternatively, the Python `help()` function can be used.

In [4]:
calculations.AimsCalculation?
#help(calculations.AimsCalculation)

We will now read in the calculations for the other compounds.

In [5]:
BaZrS3_calc = calculations.AimsCalculation("../BaZrS3/raw_aims_files/ternary/BaZrS3_Pnma/pbesol/aims.out")
ZrS2_calc =  calculations.AimsCalculation("../BaZrS3/raw_aims_files/binary/ZrS2_P-3m1/pbesol/aims.out")

## Step 3 - create `materials`

The attributes (name, stoichiometry, filepath) of a `materials.Solid` has to be stored in a variable.

In [6]:
BaZrS3 = materials.Solid('BaZrS3',{"Ba":1,"Zr":1,"S":3},"../BaZrS3/phonopy_output/BaZrS3_Pnma.dat",BaZrS3_calc)

BaS = materials.Solid('BaS',{"Ba":1,"S":1},"../BaZrS3/phonopy_output/BaS_Fm-3m.dat",BaS_calc)

ZrS2 = materials.Solid('ZrS2',{"Zr":1,"S":2},"../BaZrS3/phonopy_output/ZrS2_P-3m1.dat",ZrS2_calc)

In [7]:
vars(BaZrS3)

{'name': 'BaZrS3',
 'stoichiometry': {'Ba': 1, 'Zr': 1, 'S': 3},
 'energies': {'pbesol': -1425527.242293353},
 'N': 5,
 'volume': 484.624,
 'NAtoms': 20,
 'fu_cell': 4.0,
 'phonons': '/Users/lucy/PycharmProjects/ThermoPot/thermopot/../BaZrS3/phonopy_output/BaZrS3_Pnma.dat'}

# Step 4 - define a `reaction`

We model the composition or decomposition of our `materials` with the
`reaction` class. The first argument is a Python dictionary containing the
reactants and number of formula units for each the second argument is  a
Python dictionary containing the propducs and number of formula units for each.

In [8]:
reaction_one = reactions.Reaction({BaZrS3:1}, {BaS:1,ZrS2:1})

In [9]:
vars(reaction_one)

{'reactants': {<thermopot.materials.Solid at 0x7f90746a1700>: 1},
 'products': {<thermopot.materials.Solid at 0x7f90746a1070>: 1,
  <thermopot.materials.Solid at 0x7f90746a1610>: 1},
 'T': 298.15,
 'P': 100000.0,
 'fu_scaling': 1}

In [10]:
import numpy as np

T = np.linspace(100, 1000, 100)
P = np.array(np.logspace(1, 7, 100), ndmin=2).transpose()

reaction_one.Dmu(T,P)

In [11]:
GFE_reaction_one = reactions.Reaction

In [12]:
import numpy as np

T = np.linspace(100, 1000, 100)
P = np.array(np.logspace(1, 7, 100), ndmin=2).transpose()

GFE = GFE_reaction_one.Dmu(T,P) / 1000

AttributeError: 'numpy.ndarray' object has no attribute 'P'

In [ ]:
plots.plot_TvsP(T,P,GFE)

In [ ]:
import numpy as np
pot_1 = np.array([[3,2,1],[4,1,1],[2,1,1]])
pot_2 = np.array([[0,0,2],[0,4,5],[0,4,5]])

In [ ]:
plots.plot_TvsP(np.array([0,10,20]),np.array([0,10,20]).transpose(),pot_1,
                pot_2,material_labels=["A","B"])